In [ ]:
!pip install --upgrade google-meridian google-cloud-storage

import os
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import arviz as az
from google.cloud import storage

from meridian import constants
from meridian.data import load
from meridian.model import model, spec, prior_distribution
from meridian.analysis import optimizer, visualizer, summarizer

BUCKET_NAME = "meridian-mmm"
GCS_BASE_PATH = f"gs://{BUCKET_NAME}/outputs/results/"
CSV_FILENAME = "meridian_lite_weekly.csv"
GCS_CSV_PATH = f"gs://{BUCKET_NAME}/{CSV_FILENAME}"

timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
RESULTS_DIR = f"{GCS_BASE_PATH}{timestamp}/"
LOCAL_RESULTS_DIR = f"/content/results/{timestamp}/"
os.makedirs(LOCAL_RESULTS_DIR, exist_ok=True)

def upload_to_gcs(local_path, gcs_path):
    storage_client = storage.Client()
    bucket = storage_client.bucket(BUCKET_NAME)
    blob = bucket.blob(gcs_path)
    blob.upload_from_filename(local_path)

def generate_index_html():
    storage_client = storage.Client()
    bucket = storage_client.bucket(BUCKET_NAME)
    blobs = bucket.list_blobs(prefix="outputs/results/")

    folders = {}

    for blob in blobs:
        parts = blob.name.split("/")
        if len(parts) >= 3:
            folder = "/".join(parts[:3])
            file_name = parts[-1]
            if file_name == "index.html":
                continue
            if folder not in folders:
                folders[folder] = []
            if file_name.endswith(".html"):
                folders[folder].append(file_name)

    index_content = """
    <html>
    <head>
        <title>Resultados del Entrenamiento</title>
        <link rel="apple-touch-icon" sizes="57x57" href="https://perpetual.pe/assets/images/common/favicon/apple-touch-icon-57x57.png">
        <link rel="apple-touch-icon" sizes="60x60" href="https://perpetual.pe/assets/images/common/favicon/apple-touch-icon-60x60.png">
        <link rel="apple-touch-icon" sizes="72x72" href="https://perpetual.pe/assets/images/common/favicon/apple-touch-icon-72x72.png">
        <link rel="apple-touch-icon" sizes="76x76" href="https://perpetual.pe/assets/images/common/favicon/apple-touch-icon-76x76.png">
        <link rel="apple-touch-icon" sizes="114x114" href="https://perpetual.pe/assets/images/common/favicon/apple-touch-icon-114x114.png">
        <link rel="apple-touch-icon" sizes="120x120" href="https://perpetual.pe/assets/images/common/favicon/apple-touch-icon-120x120.png">
        <link rel="apple-touch-icon" sizes="144x144" href="https://perpetual.pe/assets/images/common/favicon/apple-touch-icon-144x144.png">
        <link rel="apple-touch-icon" sizes="152x152" href="https://perpetual.pe/assets/images/common/favicon/apple-touch-icon-152x152.png">
        <link rel="apple-touch-icon" sizes="180x180" href="https://perpetual.pe/assets/images/common/favicon/apple-touch-icon-180x180.png">
        <link rel="icon" type="image/png" sizes="192x192" href="https://perpetual.pe/assets/images/common/favicon/android-chrome-192x192.png">
        <link rel="icon" type="image/png" sizes="32x32" href="https://perpetual.pe/assets/images/common/favicon/favicon-32x32.png">
        <link rel="icon" type="image/png" sizes="16x16" href="https://perpetual.pe/assets/images/common/favicon/favicon-16x16.png">
        <link rel="manifest" href="https://perpetual.pe/assets/images/common/favicon/site.webmanifest">
        <style>
            body {
                font-family: Arial, sans-serif;
                background-color: #f4f4f4;
                text-align: center;
                margin: 0;
                padding: 0;
                display: flex;
                flex-direction: column;
                height: 100vh;
                overflow: hidden;
            }
            .header {
                background-color: #3d81f0;
                color: white;
                padding: 15px;
                text-align: center;
                font-size: 18px;
                font-weight: bold;
                position: relative;
            }
            .container {
                flex: 1;
                overflow-y: auto;
                padding: 50px;
            }
            .hidden {
                display: none;
            }
            .folder-item, .file-item {
                background: #3d81f0;
                margin: 10px 0;
                padding: 10px;
                border-radius: 5px;
                color: white;
                cursor: pointer;
                font-weight: bold;
                text-align: center;
                transition: 0.3s;
            }
            .folder-item:hover, .file-item:hover {
                background: #0056b3;
            }
            .back-button {
                position: absolute;
                left: 10px;
                top: 50%;
                transform: translateY(-50%);
                background: transparent;
                border: none;
                color: white;
                font-size: 24px;
                cursor: pointer;
                padding: 10px;
            }
            .back-button {
                left: 10px;
                top: 50%;
                transform: translateY(-50%);
            }
            .fullscreen-button {
                position: absolute;
                bottom: 20px;
                right: 20px;
                background: rgba(0, 0, 0, 0.5);
                border: none;
                border-radius: 50%;
                color: white;
                padding: 20px;
                cursor: pointer;
                font-size: 16px;
                display: flex;
                align-items: center;
                justify-content: center;
            }
            .footer {
                background-color: #f4f4f4;
                color: #000000;
                text-align: center;
                font-size: 14px;
                position: relative;
                bottom: 0;
                width: 100%;
            }
            #html-viewer-container {
                flex: 1;
                display: none;
                position: relative;
            }
            #html-viewer {
                width: 100%;
                height: 100%;
                border: none;
            }
        </style>
        <script>
            let currentView = "folders";
            function updateHeader(title) {
                document.getElementById("header-title").innerText = title;
                const backButton = document.querySelector(".back-button");

                if (title === "Resultados del Entrenamiento") {
                    backButton.style.visibility = "hidden";
                } else {
                    backButton.style.visibility = "visible";
                }
            }

            function showFiles(folderId, folderName) {
                document.getElementById("folder-list").classList.add("hidden");
                document.getElementById(folderId).classList.remove("hidden");

                document.getElementById(folderId).setAttribute("data-folder-name", folderName);

                const newTitle = `Resultados del Entrenamiento/${folderName}`;
                updateHeader(newTitle);

                currentView = folderId;
            }

            function showHtml(fileUrl, folderId, fileName) {
                document.getElementById(folderId).classList.add("hidden");
                document.getElementById("html-viewer-container").style.display = "flex";
                document.getElementById("html-viewer").src = fileUrl;

                const folderElement = document.getElementById(folderId);
                const folderName = folderElement.getAttribute("data-folder-name") || folderId;

                const safeFileName = fileName || "Archivo sin nombre";

                updateHeader(`Resultados del Entrenamiento/${folderName}/${safeFileName}`);
                currentView = `${folderId}/html`;
            }

            function goBack() {
                if (currentView === "folders") return;

                if (currentView.includes("/html")) {
                    const folderId = currentView.replace("/html", "");
                    document.getElementById("html-viewer-container").style.display = "none";
                    document.getElementById(folderId).classList.remove("hidden");

                    const folderElement = document.getElementById(folderId);
                    const folderName = folderElement.getAttribute("data-folder-name") || "Desconocido";

                    updateHeader(`Resultados del Entrenamiento/${folderName}`);

                    currentView = folderId;
                } else {
                    document.getElementById(currentView).classList.add("hidden");
                    document.getElementById("folder-list").classList.remove("hidden");
                    updateHeader("Resultados del Entrenamiento");
                    currentView = "folders";
                }
            }

            window.onload = function () {
                updateHeader("Resultados del Entrenamiento");
            };
        </script>
    </head>
    <body>
        <div class="header">
            <button class="back-button" onclick="goBack()">
                &larr;
            </button>
            <span id="header-title">Resultados del Entrenamiento</span>
        </div>
        <div class="container" id="folder-list">
    """
    for folder in sorted(folders.keys()):
        folder_id = folder.replace("/", "_")
        folder_name = folder.split("/")[-1]
        index_content += f'<div class="folder-item" onclick="showFiles(\'{folder_id}\', \'{folder_name}\')">{folder_name}</div>'

    index_content += "</div>"

    for folder, files in folders.items():
        folder_id = folder.replace("/", "_")
        index_content += f'<div id="{folder_id}" class="container hidden">'
        for file in files:
            file_url = f"https://storage.cloud.google.com/{BUCKET_NAME}/{folder}/{file}"
            index_content += f'<div class="file-item" onclick="showHtml(\'{file_url}\', \'{folder_id}\', \'{file}\')">{file}</div>'
        index_content += "</div>"

    index_content += """
        <div id="html-viewer-container" class="hidden">
            <iframe id="html-viewer"></iframe>
            <button class="fullscreen-button" onclick="document.getElementById('html-viewer').requestFullscreen()">&#x26F6;</button>
        </div>
        <footer class="footer">
          <p>Copyright &copy; 2025 <a href="mailto:consulting@perpetualtech.ai">Perpetual Technologies</a>. All Rights Reserved.</p>
        </footer>
    </body>
    </html>
    """
    index_path = "/content/index.html"
    with open(index_path, "w") as f:
        f.write(index_content)
    upload_to_gcs(index_path, "outputs/results/index.html")

coord_to_columns = load.CoordToColumns(
    time='time',
    kpi='conversions',
    controls=['GQV'],
    media=['Display&DV360_impression', 'Google_Search_impression', 'Meta_Ads_impression',
           'Newspaper_impression', 'OOH_impression', 'Radio_impression', 'TikTok_Ads_impression',
           'TV_&_Offline_impression', 'YouTube_Ads_impression'],
    media_spend=['Display&DV360_spend', 'Google_Search_spend', 'Meta_Ads_spend',
                 'Newspaper_spend', 'OOH_spend', 'Radio_spend', 'TikTok_Ads_spend',
                 'TV_&_Offline_spend', 'YouTube_Ads_spend']
)

correct_media_to_channel = {media: media.split("_")[0] for media in coord_to_columns.media}
correct_media_spend_to_channel = {spend: spend.split("_")[0] for spend in coord_to_columns.media_spend}

loader = load.CsvDataLoader(
    csv_path=GCS_CSV_PATH,
    kpi_type='non_revenue',
    coord_to_columns=coord_to_columns,
    media_to_channel=correct_media_to_channel,
    media_spend_to_channel=correct_media_spend_to_channel,
)
data = loader.load()

prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(0.2, 0.9, name=constants.ROI_M)
)
model_spec = spec.ModelSpec(prior=prior)
mmm = model.Meridian(input_data=data, model_spec=model_spec)

mmm.sample_prior(500)
mmm.sample_posterior(n_chains=7, n_adapt=500, n_burnin=500, n_keep=1000)

summary_html = os.path.join(LOCAL_RESULTS_DIR, "summary_output.html")
optimization_html = os.path.join(LOCAL_RESULTS_DIR, "optimization_output.html")
model_pkl = os.path.join(LOCAL_RESULTS_DIR, "saved_mmm.pkl")

start_date = '2022-01-25'
end_date = '2024-12-31'

mmm_summarizer = summarizer.Summarizer(mmm)
mmm_summarizer.output_model_results_summary(summary_html, LOCAL_RESULTS_DIR, start_date, end_date)

budget_optimizer = optimizer.BudgetOptimizer(mmm)
optimization_results = budget_optimizer.optimize(use_kpi=True)
optimization_results.output_optimization_summary(optimization_html, LOCAL_RESULTS_DIR)

model.save_mmm(mmm, model_pkl)

upload_to_gcs(summary_html, f"outputs/results/{timestamp}/summary_output.html")
upload_to_gcs(optimization_html, f"outputs/results/{timestamp}/optimization_output.html")
upload_to_gcs(model_pkl, f"outputs/results/{timestamp}/saved_mmm.pkl")

generate_index_html()